In [1]:
# !pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


## Data: Squad
- five columns: `'id', 'title', 'context', 'question', 'answers'`

In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("squad")
raw_datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
raw_datasets['train']['id'][0], raw_datasets['train']['title'][0]

('5733be284776f41900661182', 'University_of_Notre_Dame')

In [4]:
raw_datasets['train']['question'][0], raw_datasets['train']['answers'][0]

('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]})

In [5]:
raw_datasets['train']['context'][0] #.index('Saint Bernadette Soubirous')

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [6]:
### 有可能有多個ans
raw_datasets["validation"][2]["answers"]

{'text': ['Santa Clara, California',
  "Levi's Stadium",
  "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."],
 'answer_start': [403, 355, 355]}

## tokenizer: 將word 分割成片段並轉換為有意義的表示法

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased" # "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
context = raw_datasets["train"][1]["context"]
question = raw_datasets["train"][1]["question"]

inputs = tokenizer(question, context)
inputs

{'input_ids': [101, 1327, 1110, 1107, 1524, 1104, 1103, 10360, 8022, 4304, 4334, 136, 102, 22182, 1193, 117, 1103, 1278, 1144, 170, 2336, 1959, 119, 1335, 4184, 1103, 4304, 4334, 112, 188, 2284, 10945, 1110, 170, 5404, 5921, 1104, 1103, 6567, 2090, 119, 13301, 1107, 1524, 1104, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 171, 17506, 9538, 1110, 1103, 144, 10595, 2430, 117, 170, 14789, 1282, 1104, 8070, 1105, 9284, 119, 1135, 1110, 170, 16498, 1104, 1103, 176, 10595, 2430, 1120, 10111, 20500, 117, 1699, 1187, 1103, 6567, 2090, 25153, 1193, 1691, 1106, 2216, 17666, 6397, 3786, 1573, 25422, 13149, 1107, 8109, 119, 1335, 1103, 1322, 1104, 1103, 1514, 2797, 113, 1105, 1107, 170, 2904, 1413, 1115, 8200, 1194, 124, 11739, 1105, 1103, 3487, 17917, 114, 117

### decode：將轉換後的id mapping 回 word

In [9]:
tokenizer.decode(inputs["input_ids"])

'[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

### 因為context可能會超過model可以input的size限制，因此在tokenize的時候需要將context切開

In [10]:
inputs = tokenizer(text = question,
                   text_pair = context,
                   max_length=100, # the maximum length of context
                   truncation="only_second", # 不能truncate question
                   stride=50, # move forward 50 (overlap 50 chars)
                   return_overflowing_tokens=True)

inputs.keys(), len(inputs.input_ids) #被切成4 份

(dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping']), 4)

格式：[CLS] question [SEP] context

In [11]:
for id, input_ids in enumerate(inputs["input_ids"]):
    print(id, len(input_ids), tokenizer.decode(input_ids))

0 100 [CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the G [SEP]
1 100 [CLS] What is in front of the Notre Dame Main Building? [SEP] facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernade [SEP]
2 100 [CLS] What is in front of the Notre Dame Main Building? [SEP] of the Sacred Heart. Immediately behind the basili

 return_overflowing_tokens=True 產生，內容為input 跟切分後資料的id mapping關係

In [12]:
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0]

In [13]:
### 三個例子，各自切成四份，可以用overflow_to_sample_mapping 對照回原資料的id
inputs = tokenizer(text = raw_datasets["train"][:3]["question"],
                   text_pair = raw_datasets["train"][:3]["context"],
                   max_length=100,
                   truncation="only_second",
                   stride=50,
                   return_overflowing_tokens=True)

inputs['overflow_to_sample_mapping']

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

return_offsets_mapping=True：tells us the location of each token

In [14]:
inputs = tokenizer(text = question,
                   text_pair = context,
                   max_length=100, # the maximum length of context
                   truncation="only_second", # 不能truncate question
                   stride=50, # move forward 50 (overlap 50 chars)
                   return_overflowing_tokens=True,
                   return_offsets_mapping=True)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [15]:
inputs['offset_mapping'] ##每個token在字串中的位置

[[(0, 0),
  (0, 4),
  (5, 7),
  (8, 10),
  (11, 16),
  (17, 19),
  (20, 23),
  (24, 29),
  (30, 34),
  (35, 39),
  (40, 48),
  (48, 49),
  (0, 0),
  (0, 13),
  (13, 15),
  (15, 16),
  (17, 20),
  (21, 27),
  (28, 31),
  (32, 33),
  (34, 42),
  (43, 52),
  (52, 53),
  (54, 56),
  (56, 58),
  (59, 62),
  (63, 67),
  (68, 76),
  (76, 77),
  (77, 78),
  (79, 83),
  (84, 88),
  (89, 91),
  (92, 93),
  (94, 100),
  (101, 107),
  (108, 110),
  (111, 114),
  (115, 121),
  (122, 126),
  (126, 127),
  (128, 139),
  (140, 142),
  (143, 148),
  (149, 151),
  (152, 155),
  (156, 160),
  (161, 169),
  (170, 173),
  (174, 180),
  (181, 183),
  (183, 184),
  (185, 187),
  (188, 189),
  (190, 196),
  (197, 203),
  (204, 206),
  (207, 213),
  (214, 218),
  (219, 223),
  (224, 226),
  (226, 229),
  (229, 232),
  (233, 237),
  (238, 241),
  (242, 248),
  (249, 250),
  (250, 251),
  (251, 254),
  (254, 256),
  (257, 259),
  (260, 262),
  (263, 264),
  (264, 265),
  (265, 268),
  (268, 269),
  (269, 270),
 

In [16]:
len(inputs['offset_mapping']), len(inputs['offset_mapping'][0])

(4, 100)

## Answer Alignment: 因為將context切割，answer在切割後的位置會不同，也有可能不會出現
-> 將ans位置對應到每筆切割後的位置作為target

In [17]:
answer = raw_datasets["train"][1]["answers"]
answer

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

sequence_ids：tokenizer output 來源為function的第幾個input string

In [18]:
print(inputs.sequence_ids(0)) #tokenizer的輸入來源，0表示第一個輸入(也就是question), 1表示第二個輸入(context)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [19]:
# find the start and end of the context (the first and last '1')
sequence_ids = inputs.sequence_ids(0)

ctx_start = sequence_ids.index(1) # first occurrence
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1 # last occurrence

ctx_start, ctx_end #切分後的context內容在 [13, 98](前面為question)

(13, 98)

In [20]:
## ans 在原文中的位置
ans_start_char = answer['answer_start'][0]
ans_end_char = ans_start_char + len(answer['text'][0])
ans_start_char, ans_end_char

(188, 213)

In [21]:
## 切割後的第1筆資料當作例子
offset = inputs['offset_mapping'][0]
start_idx = 0
end_idx = 0

if (ans_start_char >= offset[ctx_start][0]) and (ans_end_char <= offset[ctx_end][1]): # ans 位置在切割後的context內
    i = ctx_start
    for start_end_char in offset[ctx_start:]:
        start, end = start_end_char
        if start == ans_start_char:
            start_idx = i

        if end == ans_end_char:
            end_idx = i
            break

        i += 1

else:
     print("target is (0, 0)")
     # nothing else to do

start_idx, end_idx

(53, 57)

In [22]:
input_ids = inputs['input_ids'][0]
input_ids[start_idx : end_idx + 1], tokenizer.decode(input_ids[start_idx : end_idx + 1])

([170, 7335, 5921, 1104, 4028], 'a copper statue of Christ')

In [23]:
def find_answer_token_idx(ctx_start, ctx_end,
                          ans_start_char, ans_end_char,
                          offset):
    start_idx = 0
    end_idx = 0

    if (ans_start_char >= offset[ctx_start][0]) and (ans_end_char <= offset[ctx_end][1]): # ans 位置在切割後的context內
        i = ctx_start
        for start_end_char in offset[ctx_start:]:
            start, end = start_end_char
            if start == ans_start_char:
                start_idx = i

            if end == ans_end_char:
                end_idx = i
                break

            i += 1
    return start_idx, end_idx

In [24]:
start_idxs = []
end_idxs = []

## ans 在原文中的位置
ans_start_char = answer['answer_start'][0]
ans_end_char = ans_start_char + len(answer['text'][0])

for i, offset in enumerate(inputs['offset_mapping']):

    #找到切割後的句子arr中context 位置
    sequence_ids = inputs.sequence_ids(i)
    # print(len(offset), len(sequence_ids))

    ctx_start = sequence_ids.index(1) # first occurrence
    ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1 # last occurrence


    new_s, new_e = find_answer_token_idx(ctx_start, ctx_end,
                                         ans_start_char, ans_end_char,
                                         offset)
    start_idxs.append(new_s)
    end_idxs.append(new_e)

start_idxs, end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

In [25]:
for i in range(len(start_idxs)):
    input_ids = inputs['input_ids'][i]
    start_idx, end_idx = start_idxs[i], end_idxs[i]

    print(input_ids[start_idx : end_idx + 1],
          tokenizer.decode(input_ids[start_idx : end_idx + 1]))

[170, 7335, 5921, 1104, 4028] a copper statue of Christ
[170, 7335, 5921, 1104, 4028] a copper statue of Christ
[101] [CLS]
[101] [CLS]


## Tokenizer func. for Training set & Val set

In [31]:
# Google used 384 for SQuAD
max_length = 384
stride = 128

def tokenize_fn_train(batch):

    questions = [q.strip() for q in batch['question']]
    inputs = tokenizer(text = questions,
                       text_pair = batch['context'],
                       max_length=max_length, # the maximum length of context
                       truncation='only_second', # 不能truncate question
                       stride=stride, # move forward 50 (overlap 50 chars)
                       return_overflowing_tokens=True,
                       return_offsets_mapping=True,
                       padding='max_length')

    offset_mapping = inputs.pop("offset_mapping")
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    answers = batch['answers']
    start_idxs, end_idxs = [], []

    for i,  offset in enumerate(offset_mapping):

        ## ans 在原文中的位置
        sample_idx = orig_sample_idxs[i] #用overflow_to_sample_mapping 對照回原資料的id
        answer = answers[sample_idx]

        ans_start_char = answer['answer_start'][0]
        ans_end_char = ans_start_char + len(answer['text'][0])


        #找到切割後的句子arr中context 位置
        sequence_ids = inputs.sequence_ids(i)
        # print(len(offset), len(sequence_ids))

        ctx_start = sequence_ids.index(1) # first occurrence
        ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1 # last occurrence

        # 找到切割後contex內ans的位置，若不包含ans, 回傳 0,0
        new_s, new_e = find_answer_token_idx(ctx_start, ctx_end,
                                             ans_start_char, ans_end_char,
                                             offset)
        start_idxs.append(new_s)
        end_idxs.append(new_e)

    inputs["start_positions"] = start_idxs
    inputs["end_positions"] = end_idxs
    return inputs

In [32]:
train_dataset = raw_datasets['train'].map(tokenize_fn_train,
                                          batched=True,
                                          remove_columns=raw_datasets['train'].column_names)
len(raw_datasets['train']), len(train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 88729)

In [33]:
def tokenize_fn_val(batch):

    questions = [q.strip() for q in batch['question']]
    inputs = tokenizer(text = questions,
                       text_pair = batch['context'],
                       max_length=max_length, # the maximum length of context
                       truncation='only_second', # 不能truncate question
                       stride=stride, # move forward 50 (overlap 50 chars)
                       return_overflowing_tokens=True,
                       return_offsets_mapping=True,
                       padding='max_length')

    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    sample_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = orig_sample_idxs[i]
        sample_ids.append(batch['id'][sample_idx]) #存原本資料的 str id

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [x if sequence_ids[j] == 1 else None for j, x in enumerate(offset)] # mask掉question的部分為None

    inputs['sample_id'] = sample_ids
    return inputs

In [35]:
val_dataset = raw_datasets['validation'].map(tokenize_fn_val,
                                          batched=True,
                                          remove_columns=raw_datasets['validation'].column_names)
len(raw_datasets['validation']), len(val_dataset)

(10570, 10822)